In [104]:
import gym
from gym import Env
from gym.spaces import Discrete,Box,Dict,Tuple,MultiBinary,MultiDiscrete

import tensorboard

#import helpers
import numpy as np
import random
import os

#import Stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
##types of spaces

In [8]:
Discrete(3).sample()

2

In [11]:
Box(0,1,shape=(3,3)).sample()

array([[0.42217064, 0.5528013 , 0.412918  ],
       [0.576677  , 0.7211167 , 0.03239008],
       [0.744463  , 0.959288  , 0.51784456]], dtype=float32)

In [15]:
Tuple((Discrete(3),Box(0,1,shape=(3,3)))).sample()

(1,
 array([[0.15574369, 0.5806547 , 0.92971927],
        [0.41837105, 0.9400347 , 0.98770237],
        [0.6252329 , 0.18121397, 0.6097918 ]], dtype=float32))

In [29]:
Dict({'height':Discrete(2), 'speed': Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([48.326447], dtype=float32))])

In [21]:
MultiBinary(4).sample()

array([0, 1, 1, 0], dtype=int8)

In [23]:
MultiDiscrete([5,2,2]).sample()

array([2, 1, 1])

In [30]:
#Building an Environment

#build agent to give us the best shower possible
#Randomly temperature
#37 and 39 degrees

In [99]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [100]:
env=ShowerEnv()

In [84]:
env.observation_space.sample()

array([74.25792], dtype=float32)

In [85]:
env.reset()

array([39.])

In [102]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-44
Episode:2 Score:-22
Episode:3 Score:-58
Episode:4 Score:-8
Episode:5 Score:-38


In [106]:
#trian model
import tensorboard


log_path=os.path.join('Training','logs')
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [107]:
model.learn(total_timesteps=20000)

Logging to Training/logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.5    |
| time/              |          |
|    fps             | 526      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -35.1       |
| time/                   |             |
|    fps                  | 914         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008403063 |
|    clip_fraction        | 0.05        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000237   |

In [108]:
#save the model

shower_path = os.path.join('Trianing','saved models','shower_model_ppo')


In [109]:
model.save(shower_path)

/Users/sampson/miniforge3/envs/tensorflow24/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'Trianing/saved models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [110]:
del model

In [111]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [113]:
evaluate_policy(model,env,n_eval_episodes=10)

(12.0, 58.787753826796276)